Fixed Width File Generator
====================

### Output

Fixed width file generated.
- **File:** output/loan_pilot_fixed_width.prn


In [1]:
#output path
fixed_width_file = 'output/loan_pilot_fixed_width.prn'
#input
config_file = 'input/layout.csv' #semicolon delimited layout file.
config_json = 'input/config.json'
layout_config = {} #Config Dictionary with field name and size.

### Input

+ Config file has the details of the field size.
    - **File:** input/config.json or input/layout.csv
    - **Values:**
        - FIELD: Field name same as in the table
        - TYPE: "REQUIRED"/"OPTIONAL"/"RESERVED"
        - FROM: Starting position
        - TO: Ending position
        - SIZE: Width of the field
+ Data is in the BQ.
    - **Project:** 'koho-staging'
    - **Table Name:** 'staging_app_koho_ca.loan_pilot'



In [2]:
#input
config_file = 'input/layout.csv' #semicolon delimited layout file.
config_json = 'input/config.json'
layout_config = {} #Config Dictionary with field name and size.

project_id = '***'
table_name = '***'

### Config file in CSV format

In [3]:
import csv
def getLayoutCSV():
    with open(config_file) as csv_file:
        csv_reader = csv.DictReader(csv_file, delimiter=';')
        line_count = 0
        for row in csv_reader:
            if line_count == 0:
                print(f'Column names are {"; ".join(row)}')
                line_count += 1
            line_count += 1
            layout_config[row["REF"]] = row
        field_count = int(row["REF"])
        print(f'Processed {line_count} lines.')

### Config file in JSON format

In [4]:
import json
def getLayoutJSON():
    with open(config_json) as csv_file:
        layout = json.load(csv_file)        
        print('Field Count: '+ str(len(layout)))
        field_count = len(layout)
        print('Field Names: '+ str(layout.keys()))
    return layout

### Fixed width Reader
Function to read fixed width file using the config file in the *input* folder. This function writes out the fixed width file to CSV format.

In [5]:
def convertPRNtoCSV():
    with open('delimited.csv', 'w') as loan_csv:
        with open('fixed_width.prn') as prn_file:
            for line in prn_file:
                for ref in layout_config:
                    loan_csv.write(line[int(layout_config[ref]['FROM'])-1:int(layout_config[ref]['TO'])].strip())

                    print(line[int(layout_config[ref]['FROM'])-1:int(layout_config[ref]['TO'])].strip(), end='')
                    if(int(ref) < field_count):
                        loan_csv.write(';')
                        print(';',end='')
                print()
                loan_csv.write('\n')
        

### Function to format values
This function checks if the values are null or not and adds the required spaces to convert the value to the required width.

In [6]:
def string_check(str, width):
    if(str == None):
        return ' '.ljust(width, ' ')
    else:
        return str.ljust(width, ' ')

### Read data 
The data is read from the provided table in BQ, which returns a dataframe.

In [7]:
import pandas_gbq as pgbq

def readDataFromBQ():
    query = "select * from {}".format(table_name)
    print('Reading data from bigquery.')
    print('Project: '+project_id)
    print('Table: '+table_name)
    loan_pilot = pgbq.read_gbq(query, project_id, dialect='standard')
    count = loan_pilot['CUSTOMER_REFERENCE_NUMBER'].count()
    print('Row Count: '+ str(count))
    print('Data Read Completed.')
    return loan_pilot

### Prepare Header and Trailer

Use the hardcoded values provided by the client:
- header_prefix
- trailer_prefix
- client_id
- process_id

Get the date time of this processing
- date_time

For trailer, also include the record count value with width 8.
- eg: 00000010

In [8]:
from datetime import datetime
today = datetime.now()
# yyyymmddhhmmss
date_time = today.strftime("%Y%m%d%H%M%S")

header_prefix = 'HDR'
trailer_prefix = 'HDR'
client_id = 'CLI'
process_id = 'CONTACTP'

def getHeader():
    header_value = header_prefix + date_time + client_id + process_id 
    return header_value

def getTrailer(record_count):
    #recordcount width is 8 here.
    trailer_value = trailer_prefix + date_time + client_id + process_id +str(record_count).rjust(8,'0')
    return trailer_value

### Write the Fixed Width File


In [9]:
def writeDataInFixedWidth(loan_pilot):
    record_count =0
    print('Starting to write in file: '+fixed_width_file)
    with open(fixed_width_file, 'w') as fw_file:
        #header
        fw_file.write(getHeader())
        fw_file.write('\n')
        for i in loan_pilot.index:
            for ref in layout_config:
                fw_file.write(string_check(loan_pilot.loc[i][layout_config[ref]['FIELD']], int(layout_config[ref]['SIZE'])))
            fw_file.write('\n')
            record_count = record_count + 1
        #trailer
        fw_file.write(getTrailer(record_count))
    print('Record Count: '+ str(record_count))
    print('Fixed Width Write Completed.')

In [10]:
layout_config = getLayoutJSON()

Field Count: 15
Field Names: dict_keys(['CUSTOMER_REFERENCE_NUMBER', 'LAST_NAME', 'FIRST_NAME', 'MIDDLE_NAME', 'SUFFIX', 'FILLER1', 'SOCIAL_INSURANCE_NUMBER', 'DATE_OF_BIRTH', 'STREET_NAME', 'CITY_NAME', 'PROVINCE', 'POSTAL_CODE', 'ACCOUNT_NUMBER', 'FILLER2', 'EQUIFAX_RESERVED_FIELD'])


In [11]:
loan_pilot = readDataFromBQ()

Reading data from bigquery.
Project: ***
Table: ***


GenericGBQException: Reason: 400 POST https://bigquery.googleapis.com/bigquery/v2/projects/***/jobs: Invalid project ID '***'. Project IDs must contain 6-63 lowercase letters, digits, or dashes. Some project IDs also include domain name separated by a colon. IDs must start with a letter and may not end with a dash.

(job ID: 05479425-f326-4744-8191-0ff935a30baa)

-----Query Job SQL Follows-----

    |    .    |
   1:select * from ***
    |    .    |

In [ ]:
writeDataInFixedWidth(loan_pilot)